<a href="https://colab.research.google.com/github/NguyenDucAnforwork/NLP_training/blob/main/phoNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-crf


In [ ]:
a = tokenizer.encode("tôi đi học", add_special_tokens=True, return_tensors='pt')

In [ ]:
print()

tensor([[  0,  70,  57, 222,   2]])


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn
from torchcrf import CRF
from torch.utils.data import Dataset , DataLoader
import numpy as np
import torch.optim as optim
from tqdm import tqdm
from itertools import cycle

In [ ]:
model = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", add_special_token=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
#@title parameters
lr = 1e-6
batch_size = 8
pos_class = 20
max_epochs = 20
ner_class = 8

In [ ]:
#@title read file
words = []
labels = []

# Đọc từng dòng từ file
with open("pos_train.txt", "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()  # Loại bỏ ký tự xuống dòng
        if line:  # Kiểm tra nếu dòng không rỗng
            word, label = line.split("\t")  # Tách từ và nhãn bằng dấu tab
            words.append(word)
            labels.append(label)

# In ra một số từ và nhãn để kiểm tra
print("Words:", words[:10])
print("Labels:", labels[:10])


Words: ['Hải_tặc', 'eo_biển', 'Malacca', '(', 'kỳ', '1', ')', ':', 'Eo_biển', 'không']
Labels: ['N', 'N', 'Np', 'CH', 'N', 'M', 'CH', 'CH', 'N', 'R']


In [ ]:
ner_sentences = []
ner_labels = []

# Đọc từng dòng từ file
with open("ner_train.txt", "r", encoding="utf-8") as file:
    sentence, label = [],[]

    for line in file:
        line = line.strip()  # Loại bỏ ký tự xuống dòng
        if line:  # Kiểm tra nếu dòng không rỗng
            word, tag = line.split("\t")  # Tách từ và nhãn bằng dấu tab
            sentence.append(word)
            label.append(tag)
        else:
            ner_sentences.append(sentence)
            ner_labels.append(label)
            sentence, label = [],[]

# In ra một số từ và nhãn để kiểm tra
print("Words:", ner_sentences[10])
print("Labels:", ner_labels[10])
print(len(ner_sentences))


Words: ['Cuộc', 'hành_trình', 'như', 'dài', 'ra', 'khi', 'chúng_tôi', 'không_thể', 'đón', 'taxi', 'đi', 'về', 'miền', 'nam', 'mà', 'phải', 'chuyển', 'sang', 'xe_buýt', ',', 'băng_băng', 'đi', 'đến', 'cuối', 'ngày', 'tới', 'Singapore', '.']
Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O']
100


In [ ]:
print(np.unique([x for ner_label in ner_labels for x in ner_label]))

['B-LOC' 'B-MISC' 'B-ORG' 'B-PER' 'I-LOC' 'I-MISC' 'I-ORG' 'O']


In [ ]:
#@title preprocessing
# Đếm số nhãn duy nhất
unique_labels = np.unique(labels)
pos_class = len(unique_labels)

# Tạo dictionary để ánh xạ từ nhãn sang chỉ mục
label_to_index = {label: i for i, label in enumerate(unique_labels)}

# Tạo danh sách tensor one-hot encoding cho mỗi nhãn
one_hot_labels = []
for label in labels:
    # Tạo tensor one-hot với tất cả giá trị ban đầu là 0
    one_hot = np.zeros(pos_class)
    # Thiết lập giá trị tại chỉ mục tương ứng với nhãn thành 1
    one_hot[label_to_index[label]] = 1
    # Chuyển đổi sang tensor và thêm vào danh sách
    one_hot_labels.append(torch.tensor(one_hot))

# In ra số nhãn duy nhất và một số tensor one-hot
print("Number of unique labels:", pos_class)
print("One-hot encoding for first label:", one_hot_labels[0])


Number of unique labels: 20
One-hot encoding for first label: tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], dtype=torch.float64)


In [ ]:
#@title unique ner labels
flat_ner_labels = [x for ner_label in ner_labels for x in ner_label]
unique_ner_labels = np.unique(flat_ner_labels)
ner_class = len(unique_ner_labels)

# Tạo dictionary để ánh xạ từ nhãn sang chỉ mục
ner_index = {label: i for i, label in enumerate(unique_ner_labels)}

# # Tạo danh sách tensor one-hot encoding cho mỗi nhãn
# one_hot_labels = []
# for label in labels:
#     # Tạo tensor one-hot với tất cả giá trị ban đầu là 0
#     one_hot = np.zeros(num_unique_labels)
#     # Thiết lập giá trị tại chỉ mục tương ứng với nhãn thành 1
#     one_hot[label_to_index[label]] = 1
#     # Chuyển đổi sang tensor và thêm vào danh sách
#     one_hot_labels.append(torch.tensor(one_hot))

print(ner_index)


{'B-LOC': 0, 'B-MISC': 1, 'B-ORG': 2, 'B-PER': 3, 'I-LOC': 4, 'I-MISC': 5, 'I-ORG': 6, 'O': 7}


In [ ]:
#@title pos model
class POS_Tagging(nn.Module):
    def __init__(self, tokenizer, model, pos_class, d_model=768):
        super(POS_Tagging, self).__init__()
        self.linear = nn.Linear(d_model, pos_class)
        self.model = model
        self.tokenizer = tokenizer

    def forward(self, token_ids):
        # token_ids = self.tokenizer.encode(token, add_special_tokens=True)  # Mã hóa token thành token IDs
        input_ids = torch.tensor(token_ids)
        if input_ids.dim() == 1:
            input_ids = input_ids.unsqueeze(0)

        with torch.no_grad():
            features = self.model(input_ids)  # Models outputs are now tuples

        embedding = features.last_hidden_state.mean(dim=1)
        return embedding, nn.Softmax(dim=-1)(self.linear(embedding))


# pos_model = POS_Tagging(tokenizer, model, num_tags).to(device)
# token = "đó"
# token = token.to(device)
# output_probs = pos_model(token)
# print(output_probs)


In [ ]:
#@title ner model
class NER(nn.Module):
    def __init__(self, model, tokenizer, pos_class, ner_class, d_model=768):
        super().__init__()
        self.tokenizer = tokenizer
        self.ner_class = ner_class
        self.model = model
        self.pos_tagging = POS_Tagging(tokenizer, model, pos_class)
        self.linear1 = nn.Linear(pos_class, d_model)
        self.linear2 = nn.Linear(2*d_model, ner_class)
        self.crf = CRF(ner_class)

    def forward(self, inputs, labels):   # tokens are already padded   inputs: encoded already, (1, max_seq_len),   labels: (1, max_seq_len)
        seq_len = len(inputs)
        emissions, tags = [], []
        for input in inputs:
            embedding, p = self.pos_tagging.to(device)(input)
            t = self.linear1.to(device)(p)
            v = torch.cat((embedding, t), dim=1)
            h = self.linear2.to(device)(v)
            emissions.append(torch.tensor(h))

        emissions = torch.stack(emissions)
        for label in labels:
            tags.append(label)
        # print(tags)
        # print(len(tags), len(tags[0]))
        # tags = torch.tensor(tags.squeeze()).unsqueeze(1)
        tags = torch.stack(tags)
        # print(tags.size())
        '''
        tags: (seq_len, batch_size)
        emissions: (seq_len, batch_size, pos_class)
        '''

        # return self.crf(emissions.to(device), tags.to(device))
        return emissions, tags



In [ ]:
#@title joint model
class MyModel(nn.Module):
    def __init__(self , model, tokenizer, pos_class, ner_class):
        super(MyModel , self).__init__()
        self.pos_model = pos_model
        self.ner = ner

    def forward(self, pos_inputs, ner_inputs, ner_labels):
        embedding, outputs = pos_model(pos_inputs.long())
        emissions, tags = ner(ner_inputs, ner_labels)
        return outputs, emissions, tags

In [ ]:
#@title pos dataset
class PosDataset(Dataset):
    def __init__(self, words, labels, max_len_sequence):
        self.words = words
        self.labels = labels
        self.max_len_sequence = max_len_sequence
        self.tokenizer = tokenizer
        self.pad_id = 0
    def __len__(self):
        return len(self.words)
    def __getitem__(self, idx):
        data = {}
        pad = torch.tensor([-1])
        word = self.tokenizer.encode(self.words[idx], add_special_tokens=True, return_tensors='pt')
        label = torch.tensor(self.labels[idx])
        if len(word) > self.max_len_sequence :
            word = word[:self.max_len_sequence]
        else :
            len_pad = self.max_len_sequence - word.size(1)
            pad_vector = torch.tensor([0]*len_pad)
            word = torch.cat((word.clone().detach().view(-1), pad_vector))
        return word, label

In [ ]:
#@title ner dataset
class NERDataset(Dataset):
    def __init__(self, tokenizer, ner_sentences, ner_labels, max_len_sequence):
        self.ner_sentences = ner_sentences
        self.ner_labels = ner_labels
        self.max_len_sequence = max_len_sequence
        self.tokenizer = tokenizer
        self.pad_id = 0

    def __len__(self):
        return len(self.ner_sentences)

    def __getitem__(self, idx):
        ner_sentence, ner_label = self.ner_sentences[idx], self.ner_labels[idx]
        max_sentence_length = self.max_len_sequence
        token_ids_list = []
        for tok in ner_sentence:
            token_ids = torch.tensor(self.tokenizer.encode(tok, add_special_tokens=True))
            if (token_ids.size(0)) > 8 :
                token_ids = token_ids[:8]
            else :
                len_pad = 8 - token_ids.size(0)
                pad_vector = torch.tensor([0]*len_pad)
                token_ids = torch.cat((token_ids.clone().detach().view(-1), pad_vector))
            token_ids_list.append(token_ids)

        if len(token_ids_list) < max_sentence_length:
            for i in range(max_sentence_length - len(token_ids_list)):
                token_ids_list.append(torch.tensor([0, 0, 0, 0, 0, 0, 0, 0]))
        else:
            token_ids_list = token_ids_list[:max_sentence_length]
        # Chuyển danh sách token_ids thành tensor
        padded_token_ids = torch.stack(token_ids_list).to(device)
        # print(padded_token_ids.size())

        # Đảm bảo ner_label là một danh sách các nhãn
        ner_label_ids = [ner_index[label] for label in ner_label]

        padded_ner_label_ids = ner_label_ids + [self.pad_id] * (max_sentence_length - len(ner_label_ids))

        padded_ner_label_ids = padded_ner_label_ids[:max_sentence_length]

        padded_ner_label_ids = torch.tensor(padded_ner_label_ids).to(device)

        return padded_token_ids, padded_ner_label_ids

# ner_dataset = NERDataset(tokenizer, ner_sentences, ner_labels, max_len_sequence=50)
# for i in range(len(ner_dataset)):
#     sentence, label = ner_dataset[i]   # __getitem__ method is called
#     print(sentence.size(), label.size())
#     break


torch.Size([50, 8]) torch.Size([50])


In [ ]:
#@title prepare for training pos model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pos_model = POS_Tagging(tokenizer, model, pos_class).to(device)
pos_dataset = PosDataset(words, one_hot_labels, 8)
pos_loader = DataLoader(pos_dataset, batch_size=batch_size, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(pos_model.parameters(), lr=lr)   # dcm same bugggggggggggg


In [ ]:
#@title prepare for training ner model
ner = NER(model, tokenizer, pos_class, ner_class).to(device)
ner_dataset = NERDataset(tokenizer, ner_sentences, ner_labels, max_len_sequence=50)
ner_loader = DataLoader(ner_dataset, batch_size=batch_size, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(pos_model.parameters(), lr=lr)

In [ ]:
#@title prepare data for training joint model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

pos_model = POS_Tagging(tokenizer, model, pos_class).to(device)
pos_dataset = PosDataset(words, one_hot_labels, 8)
pos_loader = DataLoader(pos_dataset, batch_size=batch_size, shuffle=True)

ner = NER(model, tokenizer, pos_class, ner_class).to(device)
ner_dataset = NERDataset(tokenizer, ner_sentences, ner_labels, max_len_sequence=50)
ner_loader = DataLoader(ner_dataset, batch_size=batch_size, shuffle=True)

my_model = MyModel(model, tokenizer, pos_class, ner_class)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(pos_model.parameters(), lr=lr)

In [ ]:
#@title check data
for batch in ner_loader:
    sentence, label = batch
    print(sentence.size(), label.size())
    break


torch.Size([32, 50, 8]) torch.Size([32, 50])


In [ ]:
#@title pos training
for epoch in range(max_epochs):
    pos_model.train()
    running_loss = 0.0
    for inputs, labels in pos_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        embedding, outputs = pos_model(inputs.long())
        loss = criterion((outputs), (labels))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{max_epochs}, Loss: {running_loss / len(pos_loader)}")

In [ ]:
#@title ner training
for epoch in range(max_epochs):
    ner.train()
    running_loss = 0.0
    for inputs, labels in ner_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        emissions, tags = ner(inputs, labels)  # Tính toán emissions

        # Tính toán loss từ CRF
        loss = -ner.crf(emissions, labels)  # Sử dụng CRF để tính toán loss

        # Lan truyền ngược và cập nhật tham số
        optimizer.zero_grad()  # Đặt gradient về 0
        loss.backward()  # Lan truyền ngược để tính toán gradient
        optimizer.step()  # Cập nhật các tham số dựa trên gradient tính được

        running_loss += loss.item()

    # In mất mát trung bình cho epoch
    print(f"Epoch {epoch+1}/{max_epochs}, Loss: {running_loss / len(ner_loader)}")

In [ ]:
#@title joint model training
lambda1 = 0.7
lambda2 = 1 - lambda1
num_batch = min(len(pos_loader), len(ner_loader))

for epoch in range(max_epochs):
    my_model.train()
    running_loss = 0.0
    for i in range(num_batch):
        optimizer.zero_grad()
        pos_inputs, pos_labels = pos_loader[i]
        ner_inputs, ner_labels = ner_loader[i]
        pos_inputs, pos_labels, ner_inputs, ner_labels = pos_inputs.to(device), pos_labels.to(device), ner_inputs.to(device), ner_labels.to(device)

        # Forward pass
        emissions, tags = my_model.ner(ner_inputs, ner_labels)  # Tính toán emissions
        embedding, outputs = my_model.pos_model(pos_inputs.long())

        # Tính toán loss từ CRF
        loss2 = -my_model.ner.crf(emissions, labels)  # Sử dụng CRF để tính toán loss
        loss1 = criterion((outputs), (pos_labels))

        loss = lambda1 * loss1 + lambda2 * loss2
        loss.backward()  # Lan truyền ngược để tính toán gradient
        optimizer.step()  # Cập nhật các tham số dựa trên gradient tính được

        running_loss += loss.item()

    # In mất mát trung bình cho epoch
    print(f"Epoch {epoch+1}/{max_epochs}, Loss: {running_loss / num_batch}")



TypeError: 'DataLoader' object is not subscriptable

In [ ]:
#@title backup training option
from itertools import zip_longest

#@title joint model training
lambda1 = 0.7
lambda2 = 1 - lambda1

for epoch in range(max_epochs):
    my_model.train()
    running_loss = 0.0
    for pos_data, ner_data in zip_longest(pos_loader, ner_loader):
        pos_inputs, pos_labels = pos_data if pos_data is not None else (None, None)
        ner_inputs, ner_labels = ner_data if ner_data is not None else (None, None)

        if pos_inputs is not None and ner_inputs is not None:
            pos_inputs, pos_labels, ner_inputs, ner_labels = pos_inputs.to(device), pos_labels.to(device), ner_inputs.to(device), ner_labels.to(device)

            # Forward pass
            emissions, tags = my_model.ner(ner_inputs, ner_labels)  # Tính toán emissions
            embedding, outputs = my_model.pos_model(pos_inputs.long())

            # Tính toán loss từ CRF
            loss2 = -my_model.ner.crf(emissions, ner_labels)  # Sử dụng CRF để tính toán loss
            loss1 = criterion(outputs, pos_labels)

            loss = lambda1 * loss1 + lambda2 * loss2
            optimizer.zero_grad()  # Đặt gradient về 0
            loss.backward()  # Lan truyền ngược để tính toán gradient
            optimizer.step()  # Cập nhật các tham số dựa trên gradient tính được

            running_loss += loss.item()

    # In mất mát trung bình cho epoch
    if running_loss != 0:
        print(f"Epoch {epoch+1}/{max_epochs}, Loss: {running_loss / num_batch}")


<ipython-input-61-6ff85398cce3>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx])
<ipython-input-12-c5ad0d80dffc>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(token_ids)
<ipython-input-54-e8a40682294c>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emissions.append(torch.tensor(h))


Epoch 1/20, Loss: 253.78771754480326
Epoch 2/20, Loss: 253.97604734416186
Epoch 3/20, Loss: 253.8626478353372
Epoch 4/20, Loss: 253.805952546459
Epoch 5/20, Loss: 253.6457994736158
Epoch 6/20, Loss: 253.4618034601212
Epoch 7/20, Loss: 253.86360861131777
Epoch 8/20, Loss: 253.56581575985135
Epoch 9/20, Loss: 253.82692254208604
Epoch 10/20, Loss: 253.6273654692448
Epoch 11/20, Loss: 253.4686126300922
Epoch 12/20, Loss: 253.77383320125247
Epoch 13/20, Loss: 253.6154608685237
Epoch 14/20, Loss: 253.64538769057165
Epoch 15/20, Loss: 253.73908303655114
Epoch 16/20, Loss: 253.694629948185
Epoch 17/20, Loss: 253.9365615535241
Epoch 18/20, Loss: 253.45045799888098
Epoch 19/20, Loss: 253.51430823183972
Epoch 20/20, Loss: 253.47302105999913


In [ ]:
#@title test model
inputs, labels = ner_dataset[0]
print(inputs.size(), labels.size())
ner = NER(model, tokenizer, pos_class, ner_class)
a = ner.to(device)(inputs.to(device), labels.to(device))
print(a)

In [ ]:
#@title example
import torch

# Danh sách của 50 tensor có kích thước (3,)
list_of_tensors = [torch.randn(3) for _ in range(50)]

# Chuyển đổi danh sách thành một tensor
stacked_tensor = torch.stack(list_of_tensors)

# Kiểm tra kích thước của tensor mới
print(stacked_tensor.size())  # Kết quả: torch.Size([50, 3])


torch.Size([50, 3])
